In [7]:
import warnings
import requests
from requests.auth import HTTPBasicAuth
from bs4 import BeautifulSoup
import io
import csv
import os
from urllib.parse import urljoin

from aed_models import *

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# === PARAMETRI MODELLO ===
sample_rate = 16000
duration = 1
seg_len = sample_rate * duration
threshold = 0.0020

In [9]:
# === FUNZIONI ===
def load_audio(file_like, target_length=seg_len):
    waveform, sr = torchaudio.load(file_like)
    if sr != sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=sample_rate)
        waveform = resampler(waveform)

    # Mono
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    original_length = waveform.shape[1]  # Lunghezza in campioni

    # Padding o trimming
    if original_length > target_length:
        waveform = waveform[:, :target_length]
        original_length = target_length
    else:
        pad_len = target_length - original_length
        waveform = F.pad(waveform, (0, pad_len))

    return waveform.squeeze(0), original_length  # [160000], valore reale


def run_inference(model, audio_tensor, original_length, threshold=threshold):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    model.transform_tf = model.transform_tf.to(device)

    audio_tensor = audio_tensor.unsqueeze(0).to(device)  # [1, 160000]
    feature_vector = model.preprocessing(audio_tensor)   # [1, T, 640]

    # Calcolo numero di frame reali
    hop_length = model.transform_tf.hop_length
    real_t_bins = 1 + (original_length // hop_length)
    real_vector_array_size = real_t_bins - model.frames + 1

    with torch.no_grad():
        encoded = model.encoder(feature_vector)
        bottleneck = model.bottleneck(encoded)
        reconstructed = model.decoder(bottleneck)

    # Taglio alla lunghezza effettiva in feature (T)
    feature_vector = feature_vector[:, :real_vector_array_size, :]
    reconstructed = reconstructed[:, :real_vector_array_size, :]

    # Calcolo errore
    loss = F.mse_loss(reconstructed, feature_vector).item()
    return feature_vector.cpu(), reconstructed.cpu(), loss


def analyze_in_memory(url, username, password, model):
    resp = requests.get(url, auth=HTTPBasicAuth(username, password), stream=True)
    resp.raise_for_status()
    audio_data = io.BytesIO(resp.content)

    waveform, real_samples = load_audio(audio_data)
    _, _, loss = run_inference(model, waveform, real_samples)

    return loss

def list_links(url, username, password):
    print(f"Listo i link da: {url}")
    resp = requests.get(url, auth=HTTPBasicAuth(username, password))
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    links = [a['href'] for a in soup.find_all('a', href=True)]
    print(f"Link trovati: {links}")
    return links

def gather_audio_urls(base_url, username, password):
    urls = []
    links = list_links(base_url, username, password)

    # Aggiunta 1: cerca direttamente i file FLAC in base_url
    for href in links:
        if href.lower().endswith(('.wav', '.flac')):
            urls.append(urljoin(base_url + "/", href))

    # Aggiunta 2: cerca anche nelle eventuali sottocartelle
    for href in links:
        if href.endswith('/'):  # è una cartella
            sub_url = urljoin(base_url + "/", href)
            sub_links = list_links(sub_url, username, password)
            for sub_href in sub_links:
                if sub_href.lower().endswith(('.wav', '.flac')):
                    urls.append(urljoin(sub_url, sub_href))

    return urls

def analyze_all(file_urls, username, password, model, model_type, csv_name):
    """
    Analizza direttamente una lista di URL di file audio.
    Non scandisce cartelle, ma lavora su singoli path.
    """
    results = []
    for url in file_urls:
        print(f"\nAnalizzo: {url}")
        try:
            loss = analyze_in_memory(url, username, password, model)
            status = "Anomalo" if loss > threshold else "Normale"
            print(f" --> Risultato: {status} (MSE: {loss:.6f})")
            results.append((url, status, loss))
        except Exception as e:
            print(f"Errore su {url}: {e}")
            results.append((url, "Errore", None))

    # Filtra solo anomalie
    anomalies = [r for r in results if r[1] == "Anomalo"]

    # Estrai nome file (senza estensione) per il CSV
    try:
        file_name = file_urls[0].split("/")[-1].split(".")[0]
    except IndexError:
        file_name = "unknown"

    csv_filename = f"esiti_{csv_name}/{model_type}_esito_{file_name}.csv"

    # Salva CSV
    try:
        with open(csv_filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["url", "status", "loss"])
            for url, status, loss in anomalies:
                loss_str = f"{loss:.6f}" if loss is not None else ""
                writer.writerow([url, status, loss_str])
        print(f"\nSalvati {len(anomalies)} file anomali in '{csv_filename}'")

        # Inserisce intestazione personalizzata
        prepend_metadata_to_csv(
            csv_filename=csv_filename,
            modello="NS V1.01S",
            tipo_file="FLAC",
            durata_segmento="1",
            verificato1="0",
            verificato2="1"
        )

    except Exception as e:
        print(f"Errore durante il salvataggio di {csv_filename}: {e}")



def prepend_metadata_to_csv(csv_filename, modello, tipo_file, durata_segmento, verificato1, verificato2):
    header = ["modello", "tipo_file", "durata_segmento", "verificato", "verificato"]
    metadata = [modello, tipo_file, durata_segmento, verificato1, verificato2]

    try:
        with open(csv_filename, mode='r') as original:
            original_lines = original.readlines()

        with open(csv_filename, mode='w', newline='') as modified:
            writer = csv.writer(modified)
            writer.writerow(header)
            writer.writerow(metadata)
            modified.writelines(original_lines)

        print(f"Meta-intestazione inserita in '{csv_filename}'")
    except Exception as e:
        print(f"Errore durante l'inserimento della meta-intestazione in {csv_filename}: {e}")



###=== ESECUZIONE ===###
# Carica il tuo modello
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "results_1s/best_model_1s.pth"
model = NS_1()
model.load_state_dict(torch.load(model_path, map_location=device))
model_type = '1s'
csv_name = "0808"


base_urls = [
    "https://lys-ai.it/recordings/8c69c0b3-ae12-4552-9b11-0aaa0304a06d/recordings/2025-231/device/RSP1-MIC1/RSP1-MIC1_20250819_125515.wav"]

username = "milone"
password = "Neurone-pc"

# Analizza tutti i file per ogni cartella
analyze_all(base_urls, username, password, model, model_type, csv_name)



Analizzo: https://lys-ai.it/recordings/8c69c0b3-ae12-4552-9b11-0aaa0304a06d/recordings/2025-231/device/RSP1-MIC1/RSP1-MIC1_20250819_125515.wav
 --> Risultato: Anomalo (MSE: 0.003391)

Salvati 1 file anomali in 'esiti_0808/1s_esito_RSP1-MIC1_20250819_125515.csv'
Meta-intestazione inserita in 'esiti_0808/1s_esito_RSP1-MIC1_20250819_125515.csv'


In [17]:
def load_audio(file_like, target_length=seg_len):
    waveform, sr = torchaudio.load(file_like)
    if sr != sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=sample_rate)
        waveform = resampler(waveform)

    # Mono
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    original_length = waveform.shape[1]  # Lunghezza in campioni

    # Padding o trimming
    if original_length > target_length:
        waveform = waveform[:, :target_length]
        original_length = target_length
    else:
        pad_len = target_length - original_length
        waveform = F.pad(waveform, (0, pad_len))

    return waveform.squeeze(0), original_length  # [160000], valore reale


def run_inference(model, audio_tensor, original_length, threshold=threshold):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    model.transform_tf = model.transform_tf.to(device)

    audio_tensor = audio_tensor.unsqueeze(0).to(device)  # [1, 160000]
    feature_vector = model.preprocessing(audio_tensor)   # [1, T, 640]

    # Calcolo numero di frame reali
    hop_length = model.transform_tf.hop_length
    real_t_bins = 1 + (original_length // hop_length)
    real_vector_array_size = real_t_bins - model.frames + 1

    with torch.no_grad():
        encoded = model.encoder(feature_vector)
        bottleneck = model.bottleneck(encoded)
        reconstructed = model.decoder(bottleneck)

    # Taglio alla lunghezza effettiva in feature (T)
    feature_vector = feature_vector[:, :real_vector_array_size, :]
    reconstructed = reconstructed[:, :real_vector_array_size, :]

    # Calcolo errore
    loss = F.mse_loss(reconstructed, feature_vector).item()
    return feature_vector.cpu(), reconstructed.cpu(), loss


def analyze_in_memory(url, username, password, model):
    resp = requests.get(url, auth=HTTPBasicAuth(username, password), stream=True)
    resp.raise_for_status()
    audio_data = io.BytesIO(resp.content)

    waveform, real_samples = load_audio(audio_data)
    _, _, loss = run_inference(model, waveform, real_samples)

    return loss


def list_links(url, username, password):
    print(f"Listo i link da: {url}")
    resp = requests.get(url, auth=HTTPBasicAuth(username, password))
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    links = [a['href'] for a in soup.find_all('a', href=True)]
    print(f"Link trovati: {links}")
    return links


def gather_audio_urls(base_url, username, password):
    urls = []
    links = list_links(base_url, username, password)

    # Cerca direttamente i file FLAC/WAV in base_url
    for href in links:
        if href.lower().endswith(('.wav', '.flac')):
            urls.append(urljoin(base_url + "/", href))

    # Cerca anche nelle sottocartelle
    for href in links:
        if href.endswith('/'):  # è una cartella
            sub_url = urljoin(base_url + "/", href)
            sub_links = list_links(sub_url, username, password)
            for sub_href in sub_links:
                if sub_href.lower().endswith(('.wav', '.flac')):
                    urls.append(urljoin(sub_url, sub_href))

    return urls


# 🔹 Funzione SFTP aggiornata
def upload_via_sftp(local_file, remote_path, hostname, username, password, port=2200):
    try:
        transport = paramiko.Transport((hostname, port))
        transport.connect(username=username, password=password)
        sftp = paramiko.SFTPClient.from_transport(transport)

        # Creazione ricorsiva delle cartelle remote
        remote_dir = os.path.dirname(remote_path)
        dirs = remote_dir.strip("/").split("/")
        current = ""
        for d in dirs:
            current += "/" + d
            try:
                sftp.stat(current)
            except FileNotFoundError:
                sftp.mkdir(current)

        sftp.put(local_file, remote_path)
        sftp.close()
        transport.close()
        print(f"✅ File caricato via SFTP su {remote_path}")
    except Exception as e:
        print(f"❌ Errore SFTP: {e}")


def analyze_all(file_urls, username, password, model, model_type, txt_name,
                remote_folder_path=None, remote_host=None, port=2200):
    results = []
    for url in file_urls:
        print(f"\nAnalizzo: {url}")
        try:
            loss = analyze_in_memory(url, username, password, model)
            status = "Anomalo" if loss > threshold else "Normale"
            print(f" --> Risultato: {status} (MSE: {loss:.6f})")
            results.append((url, status, loss))
        except Exception as e:
            print(f"Errore su {url}: {e}")
            results.append((url, "Errore", None))

    anomalies = [r for r in results if r[1] == "Anomalo"]

    try:
        file_name = file_urls[0].split("/")[-1].split(".")[0]
    except IndexError:
        file_name = "unknown"

    txt_filename = f"esiti_{txt_name}/{model_type}_esito_{file_name}.txt"

    # 🔹 Salva in locale come TXT
    os.makedirs(f"esiti_{txt_name}", exist_ok=True)
    with open(txt_filename, mode='w', encoding="utf-8") as file:
        file.write("### META-INFORMAZIONI ###\n")
        file.write("modello: NS V1.01S\n")
        file.write("tipo_file: FLAC\n")
        file.write("durata_segmento: 1\n")
        file.write("verificato1: 0\n")
        file.write("verificato2: 1\n")
        file.write("\n### RISULTATI ###\n")
        for url, status, loss in anomalies:
            loss_str = f"{loss:.6f}" if loss is not None else "N/A"
            file.write(f"url: {url}\nstatus: {status}\nloss: {loss_str}\n\n")

    print(f"\nTXT salvato in locale: {txt_filename}")

    # 🔹 Upload anche su server via SFTP
    if remote_folder_path and remote_host:
        remote_txt_path = f"{remote_folder_path}/{os.path.basename(txt_filename)}"
        upload_via_sftp(
            local_file=txt_filename,
            remote_path=remote_txt_path,
            hostname=remote_host,
            username=username,
            password=password,
            port=port
        )


###=== ESECUZIONE ===###
# Carica il tuo modello
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "results_1s/best_model_1s.pth"
model = NS_1()
model.load_state_dict(torch.load(model_path, map_location=device))
model_type = '1s'
txt_name = "0808"

base_urls = [
    "https://lys-ai.it/recordings/8c69c0b3-ae12-4552-9b11-0aaa0304a06d/recordings/2025-231/device/RSP1-MIC1/RSP1-MIC1_20250819_125515.wav"
]

username = "milone"
password = "Neurone-pc"

# Analizza e salva anche su server (via SFTP)
analyze_all(
    base_urls, username, password, model, model_type, txt_name,
    remote_folder_path="/mnt/BB-S_STORAGE/8c69c0b3-ae12-4552-9b11-0aaa0304a06d/recordings/2025-231/device/RSP1-MIC1/esiti_txt",
    remote_host="151.14.46.173",
    port=2200
)


Analizzo: https://lys-ai.it/recordings/8c69c0b3-ae12-4552-9b11-0aaa0304a06d/recordings/2025-231/device/RSP1-MIC1/RSP1-MIC1_20250819_125515.wav
 --> Risultato: Anomalo (MSE: 0.003391)

TXT salvato in locale: esiti_0808/1s_esito_RSP1-MIC1_20250819_125515.txt
✅ File caricato via SFTP su /mnt/BB-S_STORAGE/8c69c0b3-ae12-4552-9b11-0aaa0304a06d/recordings/2025-231/device/RSP1-MIC1/esiti_txt/1s_esito_RSP1-MIC1_20250819_125515.txt
